In [1]:
import sys
sys.path.append("/Users/zhengwenjie/AI/work/ML_3/2017-CCF-BDCI-AIJudge")

In [2]:
# -*- encoding:utf-8 -*-
import codecs
import subprocess
from collections import namedtuple

import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from utils import LOGGER
from config.db_config import Config
config = Config()

/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## 加载数据

In [3]:
data_df = pd.read_csv(config.data_csv_path,encoding='utf-8')
data_df.head()

,id,content,laws,penalty
0,16,公诉 机关 霍邱县 人民检察院 被告人 许某 1975 日生 2012 因涉嫌 危险 驾驶 ...,"133,72,73",3
1,32,公诉 机关 海口市 龙华区 人民检察院 被告人 王某 海口市 龙华区 人民检察院 海龙 检公...,"347,67,52,64",1
2,41,公诉 机关 广东省 潮州市 人民检察院 被告人 覃学彬 1980 出生 广西壮族自治区 大新...,"263,25,52,53,55,56",5
3,57,公诉 机关 榆林市 榆阳区 人民检察院 上诉人 原审 被告人 2012 因涉嫌 盗窃罪 榆林...,"264,52,53,67,72,73",5
4,60,公诉 机关 榆阳区 人民检察院 上诉人 原审 被告人 刘某 汉族 陕西省 横山县 小学文化 ...,"224,25,26,27,52,72",7


In [4]:
data_df.shape

(100, 4)

## 准备doc2vec 格式数据

In [5]:
doc_f = codecs.open(config.feat_d2v,encoding='utf-8',mode='w')
for i,content in enumerate(data_df['content']):
    if i%10000==0:
        LOGGER.log('iter=%d' %i)
    doc_f.write(u'_*{} {}\n'.format(i, content))
doc_f.close()    

2019-03-27 22:20:13 iter=0


In [7]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags')
class Doc_list(object):
    def __init__(self, f):
        self.f = f
    def __iter__(self):
        for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
            words = line.strip().split(' ')
            tags = [int(words[0][2:])]
            words = words[1:]
            yield SentimentDocument(words,tags)

In [10]:
text="_*9 公诉 机关 平湖市 人民检察院 被告人 童某 个体 拆房 因涉嫌 盗窃罪 2012 平湖市 公安局 刑事拘留 同年 依法逮捕 羁押于 本市 看守所 平湖市 人民检察院 以平检 刑诉 2012 525 起诉书 指控 被告人 某犯 盗窃罪 2012 日向 本院 提起公诉 本院 同日 受理 依法 简易程序 实行 独任 审判 公开 开庭审理 本案 被告人 童某 到庭 参加 诉讼 现已 审理 终结 2008 时许 被告人 童某 童得明 判刑 本市 曹桥 街道 愚桥 村大 竹园 废品收购 被告人 童某 路边 望风 童得明 采用 搭线 手段 得失 停放 该处 华益牌 电瓶 三轮车 一辆 价值 2475 查明 赃物 电瓶 三轮车 扣押 发还 失主 上述事实 被告人 童某 开庭审理 过程 无异议 接受 刑事案件 登记表 失主 陈述 证人 证言 同伙 供述 刑事 判决书 辨认 笔录 照片 身份证明 搜查 笔录 扣押 发还 物品 清单 价格 鉴定结论 抓获 证据 予以 证实 足以认定"

In [32]:
words = text.strip().split(' ')
tags = [int(words[0][2:])]
words = words[1:]
print(words[0:4])
print(tags)
sd = SentimentDocument(words,tags)
sd

['公诉', '机关', '平湖市', '人民检察院']
[9]


SentimentDocument(words=['公诉', '机关', '平湖市', '人民检察院', '被告人', '童某', '个体', '拆房', '因涉嫌', '盗窃罪', '2012', '平湖市', '公安局', '刑事拘留', '同年', '依法逮捕', '羁押于', '本市', '看守所', '平湖市', '人民检察院', '以平检', '刑诉', '2012', '525', '起诉书', '指控', '被告人', '某犯', '盗窃罪', '2012', '日向', '本院', '提起公诉', '本院', '同日', '受理', '依法', '简易程序', '实行', '独任', '审判', '公开', '开庭审理', '本案', '被告人', '童某', '到庭', '参加', '诉讼', '现已', '审理', '终结', '2008', '时许', '被告人', '童某', '童得明', '判刑', '本市', '曹桥', '街道', '愚桥', '村大', '竹园', '废品收购', '被告人', '童某', '路边', '望风', '童得明', '采用', '搭线', '手段', '得失', '停放', '该处', '华益牌', '电瓶', '三轮车', '一辆', '价值', '2475', '查明', '赃物', '电瓶', '三轮车', '扣押', '发还', '失主', '上述事实', '被告人', '童某', '开庭审理', '过程', '无异议', '接受', '刑事案件', '登记表', '失主', '陈述', '证人', '证言', '同伙', '供述', '刑事', '判决书', '辨认', '笔录', '照片', '身份证明', '搜查', '笔录', '扣押', '发还', '物品', '清单', '价格', '鉴定结论', '抓获', '证据', '予以', '证实', '足以认定'], tags=[9])

In [47]:
d2v = Doc2Vec(dm=0, size=100, negative=5, hs=0, min_count=3, window=30, sample=1e-5, workers=8, alpha=0.025, min_alpha=0.025)
doc_list = Doc_list(config.feat_d2v)
d2v.build_vocab(doc_list)


/anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [48]:
rows = data_df.shape[0]
print(rows)

100


In [49]:
d2v.docvecs[0]

array([-4.3182541e-03,  4.4409172e-03,  4.3915943e-03,  4.0324596e-03,
       -4.8620272e-03,  4.9612089e-03, -1.9419260e-03,  7.3655194e-04,
       -4.2478000e-03, -4.4417460e-03,  3.8351428e-03, -1.9655465e-03,
       -2.3090076e-03,  4.7950116e-03,  1.2002439e-03, -1.3877178e-03,
       -3.7086068e-03,  3.2611459e-03, -2.4280834e-03, -1.5663636e-03,
        4.9653081e-03, -4.8063262e-03, -2.3442002e-03,  2.5577985e-03,
       -4.1141521e-04,  3.9373124e-03, -3.6578628e-03, -1.8513150e-03,
        1.6956716e-03, -3.4804724e-04,  2.8502347e-03,  7.0830225e-04,
       -9.2814380e-04,  4.1841879e-03,  2.4764354e-03,  1.5086981e-03,
        1.9099091e-03,  2.6693434e-04,  2.2070396e-03,  3.0539993e-03,
        4.0080254e-03, -1.6825700e-03,  2.3504747e-03,  2.8719869e-04,
        4.6842024e-03,  4.8029842e-03, -3.4003523e-03, -1.8557182e-03,
        4.6253442e-03, -3.9783795e-03,  1.2922322e-03,  8.7766489e-04,
       -9.3204377e-04, -1.7861970e-03,  3.5240257e-03, -1.3430151e-03,
      

In [50]:
for i in range(5):
    LOGGER.log('pass:' + str(i))
    doc_list = Doc_list(config.feat_d2v)
    d2v.train(doc_list,total_examples=d2v.corpus_count,epochs=d2v.iter)
    X_d2v = np.array([d2v.docvecs[i] for i in range(rows)])
    print(X_d2v)

2019-03-27 23:03:45 pass:0
[[-3.85557190e-02 -8.46272334e-03  1.96341251e-04 ... -1.13345841e-02
  -9.68073588e-03  2.52367482e-02]
 [-5.27048185e-02 -1.61880162e-02 -1.98509591e-03 ... -1.44782839e-02
  -7.59404013e-03  4.00424898e-02]
 [-2.13096350e-01 -8.65359381e-02 -2.29048785e-02 ... -5.07480279e-02
  -3.86174396e-02  1.61995247e-01]
 ...
 [-9.26070586e-02 -3.53039280e-02 -1.32279582e-02 ... -2.16711760e-02
  -1.63207408e-02  7.14665577e-02]
 [-7.10530430e-02 -3.19509171e-02 -1.15216728e-02 ... -1.60237737e-02
  -1.32704275e-02  5.57281598e-02]
 [-1.18597209e-01 -4.48979475e-02 -1.69800799e-02 ... -2.51497049e-02
  -2.07729246e-02  7.74738565e-02]]
2019-03-27 23:03:45 pass:1
[[-0.1998142  -0.06873518 -0.01969432 ... -0.05114918 -0.03761733
   0.14026251]
 [-0.2182668  -0.0785698  -0.0220455  ... -0.0551683  -0.03627793
   0.15860102]
 [-0.43662584 -0.17747687 -0.04565908 ... -0.10452291 -0.08024776
   0.33109665]
 ...
 [-0.20471248 -0.07863896 -0.02498344 ... -0.04910282 -0.03732

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  after removing the cwd from sys.path.


[[-0.25728846 -0.0940193  -0.01923986 ... -0.06527761 -0.05291924
   0.1867334 ]
 [-0.26058567 -0.09651265 -0.02051012 ... -0.06435524 -0.04660822
   0.19562016]
 [-0.3939886  -0.19605665 -0.00976167 ... -0.08911844 -0.0932163
   0.33961254]
 ...
 [-0.26035443 -0.10178475 -0.03227943 ... -0.06171424 -0.04675297
   0.19255355]
 [-0.234942   -0.09789231 -0.02791506 ... -0.05593792 -0.04495408
   0.17551324]
 [-0.29766738 -0.11622349 -0.03623066 ... -0.06688774 -0.05428843
   0.20582312]]
2019-03-27 23:03:45 pass:3
[[-0.2844492  -0.10423835 -0.02791001 ... -0.0710094  -0.05436635
   0.19804083]
 [-0.282962   -0.10725115 -0.02751986 ... -0.06901077 -0.0459137
   0.20509048]
 [-0.31639993 -0.24059844  0.05870079 ... -0.05748492 -0.11966967
   0.36508408]
 ...
 [-0.26107803 -0.10141445 -0.03404872 ... -0.05878997 -0.04409301
   0.19124246]
 [-0.26508507 -0.11458714 -0.03917571 ... -0.06215733 -0.04624797
   0.18460791]
 [-0.31682894 -0.11960971 -0.04585588 ... -0.0686121  -0.0510013
   0.210